In software engineering, a design pattern is a general recipe for solving a common
 design problem.

Although design patterns are language independent, that does not mean every pat
tern applies to every language. For example, it doesn’t
make sense to emulate the recipe of the Iterator pattern in Python, because the pat
tern is embedded in the language and ready to use in the form of generators

In particular, in the context of languages with first-class functions, Norvig (“Design Patterns in Dynamic Languages”, Peter Norvig, 1996 presentation) suggests rethinking the classic patterns known as **Strategy, Command, Template Method, and Visitor.**

## Strategy method
Context: Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. In the ecommerce example, the context is an Order, which is configured to apply a promotional discount according to one of several algorithms.

Strategy:The interface common to the components that implement the different algorithms. In our example, this role is played by an abstract class called Promotion.

Concrete strategy: One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo, and LargeOrderPromo are the three concrete strategies implemented.

In [6]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None # This means the promotion field can either be a Promotion object or None.

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


# ABC: A base class that allows you to define abstract classes 
# (classes that are meant to be subclassed, not instantiated directly).
class Promotion(ABC):  # the Strategy: an abstract base class
    # A decorator that marks a method that must be implemented by any subclass.
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [7]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = (LineItem('banana', 4, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5)))
# The FidelityPromo promotion gives no discount to joe.
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

Often concrete strategies have no internal state; they only deal with data from the context. If that is the case, then by all means use plain old functions instead of coding single-method classes implementing a single-method interface declared in yet another class. A function is more lightweight than an instance of a user-defined class, and there is no need for Flyweight because each strategy function is created just once per Python process when it loads the module. A plain function is also “a shared object that can be used in multiple contexts simultaneously.

## Function-Oriented Strategy
 replacing the concrete strategies with simple functions
 and removing the Promo abstract class

 "functions are first-class citizens in Python," we mean that functions are treated like any other object — just like strings, integers, or lists.

In [ ]:
from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal  # use decimal when you need to be precise about the float

    def total(self):
        return self.price * self.quantity

@dataclass(frozen=True)
class Order:  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None  # <1> promotion may be None, or it may be a callable

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)  # <2> promotion is not a method. It’s an instance
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


def fidelity_promo(order: Order) -> Decimal:  # <4> Each strategy is a function.
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5))]
banana_cart = [LineItem('banana', 30, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5'))]
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [4]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

## Create a “metastrategy” that selects the best discount

In [8]:
from decimal import Decimal

promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

## Finding Strategies in a Module

In [9]:
globals().items()

dict_items([('__name__', '__main__'), ('__doc__', 'Automatically created module for IPython interactive environment'), ('__package__', None), ('__loader__', None), ('__spec__', None), ('__builtin__', <module 'builtins' (built-in)>), ('__builtins__', <module 'builtins' (built-in)>), ('_ih', ['', "joe = Customer('John Doe', 0)\nann = Customer('Ann Smith', 1100)\ncart = [LineItem('banana', 4, Decimal('.5')),\n    LineItem('apple', 10, Decimal('1.5')),\n    LineItem('watermelon', 5, Decimal(5))]\nbanana_cart = [LineItem('banana', 30, Decimal('.5')),\n    LineItem('apple', 10, Decimal('1.5'))]\nOrder(ann, cart, fidelity_promo)", 'from collections.abc import Sequence\nfrom dataclasses import dataclass\nfrom decimal import Decimal\nfrom typing import Optional, Callable, NamedTuple\n\n\nclass Customer(NamedTuple):\n    name: str\n    fidelity: int\n\n\nclass LineItem(NamedTuple):\n    product: str\n    quantity: int\n    price: Decimal\n\n    def total(self):\n        return self.price * self.

In [10]:
from decimal import Decimal

promos = [promo for name, promo in globals().items()
                if name.endswith('_promo') and
                   name != 'best_promo'
]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

In [14]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

## Decorator-Enhanced strategy pattern

In [16]:
from decimal import Decimal
from typing import Callable


Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = []


def promotion(promo: Promotion) -> Promotion:
    promos.append(promo)
    return promo


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)


@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


@promotion
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

## Logging System using Strategy Pattern

In [6]:
import datetime
import json
from typing import NamedTuple, Callable

class UserAction(NamedTuple):
    user_id: int
    action_type: str
    timestamp: datetime.datetime

def process_action(action: UserAction, logger: Callable[[UserAction], None]) -> None:
    logger(action)

def console_logger(action: UserAction) -> None:
    print(action)

def json_logger(action: UserAction) -> None:
    json_str = json.dumps(action._asdict(), default=str)  
    print(json_str)

def remote_logger(action: UserAction) -> None:
    print(f"Sending to remote server: {action._asdict()}")  # Simulated remote log

In [7]:
now = datetime.datetime.now()
action = UserAction(user_id=1, action_type="click", timestamp=now)

process_action(action, console_logger)
process_action(action, json_logger)
process_action(action, remote_logger)

UserAction(user_id=1, action_type='click', timestamp=datetime.datetime(2025, 7, 24, 9, 31, 58, 959344))
{"user_id": 1, "action_type": "click", "timestamp": "2025-07-24 09:31:58.959344"}
Sending to remote server: {'user_id': 1, 'action_type': 'click', 'timestamp': datetime.datetime(2025, 7, 24, 9, 31, 58, 959344)}
